# L1: Automated Project: Planning, Estimation, and Allocation

This notebook initial imports will load environment variable fro the .env file located at the root.  
Specifically, it will load model api keys.
Here we use a local model. No API KEY required

## Initial Imports

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
# from helper import load_env
# load_env()
from helpers import DEFAULT_LLM

import os
import yaml
from crewai import Agent, Task, Crew, LLM

print(DEFAULT_LLM.base_url, DEFAULT_LLM.model)

ollama/llama3.2 http://localhost:11430
<crewai.llm.LLM object at 0x7facc835bb10> http://localhost:11430 ollama/llama3.2


In our case we use a [local model](https://docs.crewai.com/how-to/llm-connections#using-local-models-with-ollama).  
Download [Ollama](https://ollama.com/download).  
Pull the [desired model](https://github.com/ollama/ollama?tab=readme-ov-file) :

```bash
ollama pull llama3.2
```
configure the agent:  

```python
    agent = Agent(
        role='Local AI Expert',
        goal='Process information using a local model',
        backstory="An AI assistant running on local hardware.",
        llm=LLM(model="ollama/llama3.2", base_url="http://localhost:11434")
    )
```

## Set OpenAI Model

In [1]:
# os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'

## Loading Tasks and Agents YAML files

In [2]:
# Define file paths for YAML configurations
files = {
    'agents': 'config_L1/agents.yaml',
    'tasks': 'config_L1/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

NameError: name 'yaml' is not defined

## Create Pydantic Models for Structured Output

In [ ]:
from typing import List
from pydantic import BaseModel, Field

class TaskEstimate(BaseModel):
    task_name: str = Field(..., description="Name of the task")
    estimated_time_hours: float = Field(..., description="Estimated time to complete the task in hours")
    required_resources: List[str] = Field(..., description="List of resources required to complete the task")

class Milestone(BaseModel):
    milestone_name: str = Field(..., description="Name of the milestone")
    tasks: List[str] = Field(..., description="List of task IDs associated with this milestone")

class ProjectPlan(BaseModel):
    tasks: List[TaskEstimate] = Field(..., description="List of tasks with their estimates")
    milestones: List[Milestone] = Field(..., description="List of project milestones")

## Create Crew, Agents and Tasks

In [ ]:
# default_llm = LLM(model="ollama/llama3.2", base_url="http://localhost:11434")
default_llm = DEFAULT_LLM  # LLM(model="ollama/llama3.2")

# Creating Agents
project_planning_agent = Agent(
  config=agents_config['project_planning_agent'],
  llm=default_llm,
)

estimation_agent = Agent(
  config=agents_config['estimation_agent'],
  llm=default_llm,
)

resource_allocation_agent = Agent(
  config=agents_config['resource_allocation_agent'],
  llm=default_llm,
)

# Creating Tasks
task_breakdown = Task(
  config=tasks_config['task_breakdown'],
  agent=project_planning_agent,
)

time_resource_estimation = Task(
  config=tasks_config['time_resource_estimation'],
  agent=estimation_agent
)

resource_allocation = Task(
  config=tasks_config['resource_allocation'],
  agent=resource_allocation_agent,
  output_pydantic=ProjectPlan # This is the structured output we want
)

# Creating Crew
crew = Crew(
  agents=[
    project_planning_agent,
    estimation_agent,
    resource_allocation_agent
  ],
  tasks=[
    task_breakdown,
    time_resource_estimation,
    resource_allocation
  ],
  verbose=True
)

ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable



Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers



ERROR:opentelemetry.sdk.trace.export:Exception while exporting Span batch.
Traceback (most recent call last):
  File "/home/debian/demo/mlebreuil-crewai-testing/.venv/lib/python3.11/site-packages/urllib3/connectionpool.py", line 787, in urlopen
    response = self._make_request(
               ^^^^^^^^^^^^^^^^^^^
  File "/home/debian/demo/mlebreuil-crewai-testing/.venv/lib/python3.11/site-packages/urllib3/connectionpool.py", line 488, in _make_request
    raise new_e
  File "/home/debian/demo/mlebreuil-crewai-testing/.venv/lib/python3.11/site-packages/urllib3/connectionpool.py", line 464, in _make_request
    self._validate_conn(conn)
  File "/home/debian/demo/mlebreuil-crewai-testing/.venv/lib/python3.11/site-packages/urllib3/connectionpool.py", line 1093, in _validate_conn
    conn.connect()
  File "/home/debian/demo/mlebreuil-crewai-testing/.venv/lib/python3.11/site-packages/urllib3/connection.py", line 741, in connect
    sock_and_verified = _ssl_wrap_socket_and_match_hostname(
   

## Crew's Inputs

In [ ]:
from IPython.display import display, Markdown

project = 'Website'
industry = 'Technology'
project_objectives = 'Create a website for a small business'
team_members = """
- John Doe (Project Manager)
- Jane Doe (Software Engineer)
- Bob Smith (Designer)
- Alice Johnson (QA Engineer)
- Tom Brown (QA Engineer)
"""
project_requirements = """
- Create a responsive design that works well on desktop and mobile devices
- Implement a modern, visually appealing user interface with a clean look
- Develop a user-friendly navigation system with intuitive menu structure
- Include an "About Us" page highlighting the company's history and values
- Design a "Services" page showcasing the business's offerings with descriptions
- Create a "Contact Us" page with a form and integrated map for communication
- Implement a blog section for sharing industry news and company updates
- Ensure fast loading times and optimize for search engines (SEO)
- Integrate social media links and sharing capabilities
- Include a testimonials section to showcase customer feedback and build trust
"""

# Format the dictionary as Markdown for a better display in Jupyter Lab
formatted_output = f"""
**Project Type:** {project}

**Project Objectives:** {project_objectives}

**Industry:** {industry}

**Team Members:**
{team_members}
**Project Requirements:**
{project_requirements}
"""
# Display the formatted output as Markdown
display(Markdown(formatted_output))


**Project Type:** Website

**Project Objectives:** Create a website for a small business

**Industry:** Technology

**Team Members:**

- John Doe (Project Manager)
- Jane Doe (Software Engineer)
- Bob Smith (Designer)
- Alice Johnson (QA Engineer)
- Tom Brown (QA Engineer)

**Project Requirements:**

- Create a responsive design that works well on desktop and mobile devices
- Implement a modern, visually appealing user interface with a clean look
- Develop a user-friendly navigation system with intuitive menu structure
- Include an "About Us" page highlighting the company's history and values
- Design a "Services" page showcasing the business's offerings with descriptions
- Create a "Contact Us" page with a form and integrated map for communication
- Implement a blog section for sharing industry news and company updates
- Ensure fast loading times and optimize for search engines (SEO)
- Integrate social media links and sharing capabilities
- Include a testimonials section to showcase customer feedback and build trust



## Kicking off the crew

In [ ]:
# The given Python dictionary
inputs = {
  'project_type': project,
  'project_objectives': project_objectives,
  'industry': industry,
  'team_members': team_members,
  'project_requirements': project_requirements
}

# Run the crew
result = crew.kickoff(
  inputs=inputs
)

ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:LiteLLM call failed: litellm.BadRequestError: LLM Provider NOT provided. Pass in the LLM provider you are trying to call. You passed model=deepseek-r1:70b
 Pass model as E.g. For 'Huggingface' inference endpoints pass in `completion(model='huggingface/starcoder',..)` Learn more: https://docs.litellm.ai/docs/providers



Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers

# Agent: The Ultimate Project Planner
## Task: Carefully analyze the project_requirements for the Website project and break them down into individual tasks. Define each task's scope in detail, set achievable timelines, and ensure that all dependencies are accounted for:

- Create a responsive design that works well on desktop and mobile devices
- Implement a modern, visually appealing user interface with a clean look
- Develop a user-friendly navigation system with intuitive menu structure
- Include an "About Us" page highlighting the company's history and values
- Design a "Services" page showcasing the business's offerings with descriptions
- Create a "Contact Us" page with a form and integrated map for communication
- Implement a blog section for sharing industry

BadRequestError: litellm.BadRequestError: LLM Provider NOT provided. Pass in the LLM provider you are trying to call. You passed model=deepseek-r1:70b
 Pass model as E.g. For 'Huggingface' inference endpoints pass in `completion(model='huggingface/starcoder',..)` Learn more: https://docs.litellm.ai/docs/providers

## Usage Metrics and Costs

Let’s see how much it would cost each time if this crew runs at scale.

In [ ]:
import pandas as pd

costs = 0.150 * (crew.usage_metrics.prompt_tokens + crew.usage_metrics.completion_tokens) / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([crew.usage_metrics.dict()])
df_usage_metrics

Total costs: $0.0009


,total_tokens,prompt_tokens,cached_prompt_tokens,completion_tokens,successful_requests
0,6263,3537,0,2726,3


## Result

In [ ]:
result.pydantic.dict()

{'tasks': [{'task_name': 'Responsive Design',
   'estimated_time_hours': 40.0,
   'required_resources': ['2 developers', '1 designer']},
  {'task_name': 'Modern User Interface',
   'estimated_time_hours': 80.0,
   'required_resources': ['3 developers', '1 designer']},
  {'task_name': 'Navigation System',
   'estimated_time_hours': 30.0,
   'required_resources': ['2 developers', '1 QA Engineer']},
  {'task_name': 'About Us Page',
   'estimated_time_hours': 20.0,
   'required_resources': ['1 developer', '1 writer']},
  {'task_name': 'Services Page',
   'estimated_time_hours': 30.0,
   'required_resources': ['2 developers', '1 writer']},
  {'task_name': 'Contact Us Page',
   'estimated_time_hours': 30.0,
   'required_resources': ['2 developers', '1 writer']},
  {'task_name': 'Blog Section',
   'estimated_time_hours': 60.0,
   'required_resources': ['3 developers', '1 writer']},
  {'task_name': 'SEO Optimization',
   'estimated_time_hours': 40.0,
   'required_resources': ['2 developers', '

## Inspect further

In [ ]:
tasks = result.pydantic.dict()['tasks']
df_tasks = pd.DataFrame(tasks)

# Display the DataFrame as an HTML table
df_tasks.style.set_table_attributes('border="1"').set_caption("Task Details").set_table_styles(
    [{'selector': 'th, td', 'props': [('font-size', '120%')]}]
)

,task_name,estimated_time_hours,required_resources
0,Responsive Design,40.000000,"['2 developers', '1 designer']"
1,Modern User Interface,80.000000,"['3 developers', '1 designer']"
2,Navigation System,30.000000,"['2 developers', '1 QA Engineer']"
3,About Us Page,20.000000,"['1 developer', '1 writer']"
4,Services Page,30.000000,"['2 developers', '1 writer']"
5,Contact Us Page,30.000000,"['2 developers', '1 writer']"
6,Blog Section,60.000000,"['3 developers', '1 writer']"
7,SEO Optimization,40.000000,"['2 developers', '1 SEO Specialist']"
8,Social Media Integration,20.000000,"['1 developer', '1 social media Specialist']"
9,Testimonials Section,30.000000,"['2 developers', '1 writer']"


### Inspecting Milestones

In [ ]:
milestones = result.pydantic.dict()['milestones']
df_milestones = pd.DataFrame(milestones)

# Display the DataFrame as an HTML table
df_milestones.style.set_table_attributes('border="1"').set_caption("Task Details").set_table_styles(
    [{'selector': 'th, td', 'props': [('font-size', '120%')]}]
)

,milestone_name,tasks
0,Responsive Design Completion,['Responsive Design']
1,Modern User Interface Completion,['Modern User Interface']
2,Navigation System Completion,['Navigation System']
3,SEO Optimization Completion,['SEO Optimization']
